In [1]:
import numpy as np
import pandas as pd
import math
import itertools
import matplotlib.pyplot as plt
import sys
import torch  
import numpy as np  
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import pandas as pd


import import_ipynb
from Environments import Square_Crossroads

importing Jupyter notebook from Environments.ipynb
Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [5, 10], 0, 0], 2: [[5, 10], [0, 5], 0, 0], 3: [[10, 5], [0, 5], 0, 0]}
Car 0 is taking a step in the environment with these actions: 
 acceleration = 0.3528283602752257, curvature = 0.9977178362096486
Car 1 is taking a step in the environment with these actions: 
 acceleration = 0.9742321620569147, curvature = 0.000592288922312576
Car 2 is taking a step in the environment with these actions: 
 acceleration = 0.2279086446790946, curvature = 0.004829066896684397
A car tried to go outside the boundaries
Car 3 is taking a step in the environment with these actions: 
 acceleration = 0.9956378522942915, curvature = 0.00031969397664568925
A car tried to go outside the boundaries
{0: [[0.03527920876150809, 4.99949408764892], [5, 10], 0.035282836027522566, 359.1784210354735], 1: [[5.097422541586141, 0.00036255586803496257], [5, 10], 0.09742321620569147, 0.21322401203252736],

 acceleration = 0.1066124139327928, curvature = 2.3274218207867377e-06
Car 3 is taking a step in the environment with these actions: 
 acceleration = 0.5243955479094345, curvature = 0.022692509837262405
A car tried to go outside the boundaries
{0: [[2.5949045909979116, 4.368764708617733], [5, 10], 0.7061217831856398, 0.0002825005866108275], 1: [[7.232703523271134, 0.4161508976179736], [5, 10], 0.6664266904135768, 38.416703404718106], 2: [[5.785033581018044, 9.741433362191907], [0, 5], 0.4121091375012646, 0.0008378718554832255], 3: [[9.974920743859867, 5.4280095968183595], [0, 5], 0.5057048063581988, 8.169303541414466]}
-6
Car 0 is taking a step in the environment with these actions: 
 acceleration = 0.5970189854122899, curvature = 5.549007176723438e-10
Car 1 is taking a step in the environment with these actions: 
 acceleration = 0.9999626469173999, curvature = 0.9851836504055397
Car 2 is taking a step in the environment with these actions: 
 acceleration = 0.7096744346315752, curvatur

In [2]:
n_times_steps = 3
seed = 10
dist_cars = 0.5

env = Square_Crossroads(n_times_steps, seed, dist_cars)


action_space = {0:[0,0], 1:[0,0], 2:[0,0], 3:[0,0]}
total_reward = 0

print(env.states)


Environment reset with param
{0: [[0, 5], [5, 10], 0, 0], 1: [[5, 0], [5, 10], 0, 0], 2: [[5, 10], [0, 5], 0, 0], 3: [[10, 5], [0, 5], 0, 0]}


In [3]:
state_tensor = torch.zeros([4,2], dtype=torch.float32)
action_tensor = torch.zeros([4,2], dtype=torch.float32)

In [4]:
class Actor_Critic(nn.Module):
    def __init__(self, state_tensor, hidden_size_actor, hidden_size_critic, learning_rate=3e-4):
        super(Actor_Critic, self).__init__()
      
        self.num_inputs = state_tensor.shape[1]
        
        
        self.critic_linear1 = nn.Linear(self.num_inputs, hidden_size_critic)
        self.critic_linear2 = nn.Linear(hidden_size_critic, 1)
        
        self.actor_linear1 = nn.Linear(self.num_inputs, hidden_size_actor)
        self.actor_linear2 = nn.Linear(hidden_size_actor, hidden_size_actor)
        self.alpha = nn.Linear(hidden_size_actor, 1)
        self.beta = nn.Linear(hidden_size_actor, 1)
        

    def forward(self, state_tensor):

        value = F.relu(self.critic_linear1(state_tensor))
        value = self.critic_linear2(value)
        
        policy_dist = F.relu(self.actor_linear1(state_tensor))
        policy_dist = F.relu(self.actor_linear2(policy_dist))

        policy_dist_alpha = torch.clamp(self.alpha(policy_dist), 0, 1)
        policy_dist_beta = torch.clamp(self.beta(policy_dist), 0, 1)
        
        
        return value, policy_dist_alpha, policy_dist_beta

In [5]:
ac = Actor_Critic(state_tensor, 20, 400, learning_rate=3e-4)

In [6]:
ac

Actor_Critic(
  (critic_linear1): Linear(in_features=2, out_features=400, bias=True)
  (critic_linear2): Linear(in_features=400, out_features=1, bias=True)
  (actor_linear1): Linear(in_features=2, out_features=20, bias=True)
  (actor_linear2): Linear(in_features=20, out_features=20, bias=True)
  (alpha): Linear(in_features=20, out_features=1, bias=True)
  (beta): Linear(in_features=20, out_features=1, bias=True)
)

In [7]:
value, policy_dist_alpha, policy_dist_beta = ac(state_tensor)

In [8]:
policy_dist_alpha

tensor([[0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ClampBackward>)

In [9]:
policy_dist_beta

tensor([[0.0654],
        [0.0654],
        [0.0654],
        [0.0654]], grad_fn=<ClampBackward>)

In [10]:
value

tensor([[-0.0856],
        [-0.0856],
        [-0.0856],
        [-0.0856]], grad_fn=<AddmmBackward>)